In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model,Sequential
from tensorflow.keras.layers import Dense,Dropout,MaxPooling2D,Conv2D,GlobalAveragePooling2D,RandomFlip,RandomRotation,RandomBrightness,RandomContrast,RandomZoom,Input,Rescaling
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard
from PIL import Image
import os


In [2]:
data='PlantVillage'
data.class
data.endswith

SyntaxError: invalid syntax (2885362014.py, line 2)

In [ ]:
classnames = []
for fname in os.listdir():
    if fname.endswith(".ipynb"):
        print(fname)
    else:
        classnames.append(fname)

In [4]:
classnames

['Pepper__bell___Bacterial_spot',
 'Pepper__bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_healthy',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_mosaic_virus',
 'Tomato__Tomato_YellowLeaf__Curl_Virus']

In [5]:
img_skip=0
for folder_name in classnames:
    folder_path=os.path.join("./",folder_name)
    print(folder_path)
    for f_name in os.listdir(folder_path):
        f_path =os.path.join(folder_path,f_name)
        try:
            img_bytes=tf.io.read_file(f_path)
            tf.io.decode_image(img_bytes)
           
        except Exception as e:
            print(f"Bad file found and deleted on {f_path}")
            img_skip +=1
            os.remove(f_path)

print(f"picture deleted due to corrupt {img_skip}")

./Pepper__bell___Bacterial_spot
./Pepper__bell___healthy
./Potato___Early_blight
./Potato___healthy
./Potato___Late_blight
./Tomato_Bacterial_spot
./Tomato_Early_blight
./Tomato_healthy
./Tomato_Late_blight
./Tomato_Leaf_Mold
./Tomato_Septoria_leaf_spot
./Tomato_Spider_mites_Two_spotted_spider_mite
./Tomato__Target_Spot
./Tomato__Tomato_mosaic_virus
./Tomato__Tomato_YellowLeaf__Curl_Virus
picture deleted due to corrupt 0


In [6]:
directory ="../PlantVillage"

In [7]:
directory

'../PlantVillage'

In [8]:

train_data =tf.keras.utils.image_dataset_from_directory(
    directory,
    subset='training',
    seed=2345,
    validation_split=0.2,
    image_size=(224,224),
    batch_size=40
)
val_data =tf.keras.utils.image_dataset_from_directory(
    directory,
    subset='validation',
    seed=2345,
    validation_split=0.2,
    image_size=(224,224),
    batch_size=40
)

Found 20638 files belonging to 15 classes.
Using 16511 files for training.
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.


In [9]:
data_aug=Sequential([
     RandomFlip('horizontal'),
    RandomRotation(0.3),
    RandomZoom(height_factor=0.3),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2)
])

In [10]:
auto_tune=tf.data.AUTOTUNE
training_ds =train_data.cache().prefetch(buffer_size=auto_tune)
validation_ds =val_data.cache().prefetch(buffer_size=auto_tune)

In [13]:
base_model.trainable=False

In [12]:
 base_model =tf.keras.applications.MobileNetV2(
  input_shape=(224,224,3),
  include_top=False,
  weights='imagenet'
  )

In [14]:
model=Sequential([
    Input(shape=(224,224,3)),
    data_aug,
    Rescaling(1./127.5,offset=1),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(124,activation='relu'),
    Dropout(0.3),
    Dense(4,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 124)            │       158,844 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 124)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,417,328 (9.22 MB)

 Trainable params: 159,344 (622.44 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[EarlyStopping(monitor='val_accuracy',patience=3,mode='max'),
               ModelCheckpoint('tomato_pest_best_model.keras',save_best_only=True,monitor='val_loss',mode='min'),
               ReduceLROnPlateau( monitor='val_loss',factor=0.2,patience=5,min_lr=1e-7,mode='min'),
               TensorBoard(log_dir='logs', histogram_freq=1,write_graph=True,write_images=False,update_freq='epoch',profile_batch=2,embeddings_freq=0,embeddings_metadata=None)]
)

In [ ]:
EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard